## Flights API

In [1]:
import requests
import pandas as pd 

from datetime import datetime
from datetime import timedelta

In [2]:
API_key = "our key"
    
from datetime import datetime
from datetime import timedelta
    
airport_icoa = "EDDB"
     
#Add 1 day.
tomorrows_date = (datetime.now() + timedelta(days=1))
tomorrows_date_from=tomorrows_date.strftime("%Y-%m-%dT%H:%M")
#add 11 hours
future_date_and_time = tomorrows_date  + timedelta(hours=11)
tomorrows_date_to = future_date_and_time.strftime("%Y-%m-%dT%H:%M")

url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{tomorrows_date_from}/{ tomorrows_date_to}"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

headers = {
 "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
 "X-RapidAPI-Key": API_key
}

response = requests.request("GET", url, headers=headers, params=querystring)

In [3]:
response

<Response [200]>

In [4]:
#option 1

arrivals_berlin = response.json()['arrivals']

def get_flight_info(flight_json):
        # terminal
        try: terminal = flight_json['arrival']['terminal']
        except: terminal = None
        # aircraft
        try: aircraft = flight_json['aircraft']['model']
        except: aircraft = None
    
        return {
            'dep_airport':flight_json['departure']['airport']['name'],
            'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
            'terminal':terminal,
            'status':flight_json['status'],
            'aircraft':aircraft,
            'icao_code':airport_icoa
        }



# [get_flight_info(flight) for flight in arrivals_berlin]
arrivals_berlin = pd.DataFrame([get_flight_info(flight) for flight in arrivals_berlin])
arrivals_berlin    

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Vienna,2022-04-08 14:20+02:00,1,Unknown,Airbus A320,EDDB
1,Copenhagen,2022-04-08 14:25+02:00,1,Unknown,Airbus A320,EDDB
2,Málaga,2022-04-08 14:25+02:00,1,Unknown,Airbus A320,EDDB
3,Sofia,2022-04-08 14:30+02:00,1,Unknown,Embraer 190,EDDB
4,London,2022-04-08 14:35+02:00,1,Unknown,Airbus A320,EDDB
...,...,...,...,...,...,...
140,Madrid,2022-04-08 22:40+02:00,1,Unknown,Airbus A320,EDDB
141,Paris,2022-04-08 22:40+02:00,1,Unknown,Airbus A220-300,EDDB
142,Treviso,2022-04-08 22:30+02:00,0,Unknown,Boeing 737-800,EDDB
143,Tel Aviv Yafo,2022-04-08 22:30+02:00,0,Unknown,Boeing 737-800,EDDB


In [5]:
### Option 2

import pandas as pd
arrivals = pd.json_normalize(response.json()['arrivals'])
# next step: select the columns you want to incude on your database
(
arrivals
    .filter(['departure.airport.name','arrival.scheduledTimeLocal',
             'arrival.terminal','status','aircraft.model'])
    .assign(icao_code = airport_icoa)
)

,departure.airport.name,arrival.scheduledTimeLocal,arrival.terminal,status,aircraft.model,icao_code
0,Vienna,2022-04-08 14:20+02:00,1,Unknown,Airbus A320,EDDB
1,Copenhagen,2022-04-08 14:25+02:00,1,Unknown,Airbus A320,EDDB
2,Málaga,2022-04-08 14:25+02:00,1,Unknown,Airbus A320,EDDB
3,Sofia,2022-04-08 14:30+02:00,1,Unknown,Embraer 190,EDDB
4,London,2022-04-08 14:35+02:00,1,Unknown,Airbus A320,EDDB
...,...,...,...,...,...,...
140,Madrid,2022-04-08 22:40+02:00,1,Unknown,Airbus A320,EDDB
141,Paris,2022-04-08 22:40+02:00,1,Unknown,Airbus A220-300,EDDB
142,Treviso,2022-04-08 22:30+02:00,0,Unknown,Boeing 737-800,EDDB
143,Tel Aviv Yafo,2022-04-08 22:30+02:00,0,Unknown,Boeing 737-800,EDDB


## Update data into database

#### Establish the connection

In [6]:
pip install SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install PyMySQL

Note: you may need to restart the kernel to use updated packages.


#### Update data in AWS DATABASE Instance

In [8]:
schema="gans"
host="you host name"
user="your username"
password="your password"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [9]:
import numpy as np
(
arrivals_berlin
    .replace({np.nan},'unknown')
    .assign(sched_arr_loc_time = lambda x: pd.to_datetime(x['sched_arr_loc_time']))
    .to_sql('arrivals', if_exists='append', con=con, index=False))